In [4]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
import nibabel as nib
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.metrics import roc_curve, auc
import scipy
import statsmodels
from statsmodels.stats.contingency_tables import mcnemar
from scipy.stats import ttest_rel, wilcoxon, shapiro, bartlett, levene
from matplotlib.collections import LineCollection

In [5]:
print(tf.__version__)
print(keras.__version__)
print(scipy.__version__)
print(statsmodels.__version__)

2.7.0
2.7.0
1.7.3
0.14.0


In [9]:
#data for binary classification
img_height = 128
filenames = os.listdir('C:/Users/Oona/Documents/Tpc/thr/covid/normal/')
filenames1 = os.listdir('C:/Users/Oona/Documents/Tpc/thr/covid/cvd')
imgs=[]
for i in range(3000):
    img_path='C:/Users/Oona/Documents/Tpc/thr/covid/normal/{}'.format(filenames[i])
    img=cv2.imread(img_path)
    img=cv2.resize(img[:,:,0],(img_height,img_height))
    imgs.append(img)
    img_path1='C:/Users/Oona/Documents/Tpc/thr/covid/cvd/{}'.format(filenames1[i])
    img1=cv2.imread(img_path1)
    img1=cv2.resize(img1[:,:,0],(img_height,img_height))
    imgs.append(img1)
x_train=np.array(imgs[0:2400])
y_train=np.array([0,1]*1200)
x_test=np.array(imgs[2400:3000])
y_test=np.array([0,1]*300)

In [5]:
#data for multi-class classification
img_height = 128
filenames0 = os.listdir('C:/Users/Oona/Documents/Tpc/thr/covid/normal/')
filenames1 = os.listdir('C:/Users/Oona/Documents/Tpc/thr/covid/cvd')
filenames2 = os.listdir('C:/Users/Oona/Documents/Tpc/thr/pneumonia/ill')
filenames3 = os.listdir('C:/Users/Oona/Documents/Tpc/thr/tuberculosis/tb')
imgs=[]
for i in range(700):
    img_path='C:/Users/Oona/Documents/Tpc/thr/covid/normal/{}'.format(filenames[i])
    img=cv2.imread(img_path)
    img=cv2.resize(img[:,:,0],(img_height,img_height))
    imgs.append(img)
    img_path1='C:/Users/Oona/Documents/Tpc/thr/covid/cvd/{}'.format(filenames1[i])
    img1=cv2.imread(img_path1)
    img1=cv2.resize(img1[:,:,0],(img_height,img_height))
    imgs.append(img1)
    img_path2='C:/Users/Oona/Documents/Tpc/thr/pneumonia/ill/{}'.format(filenames2[i])
    img2=cv2.imread(img_path2)
    img2=cv2.resize(img2[:,:,0],(img_height,img_height))
    imgs.append(img2)
    img_path3='C:/Users/Oona/Documents/Tpc//thr/tuberculosis/tb/{}'.format(filenames3[i])
    img3=cv2.imread(img_path3)
    img3=cv2.resize(img3[:,:,0],(img_height,img_height))
    imgs.append(img3)

def divideImgsIntoSets(imgs,k):

    numberOfImgs=len(imgs)
    testSetSize=int(numberOfImgs/5)
    x_train=np.array(imgs[0:(k*testSetSize)]+imgs[((k+1)*testSetSize):numberOfImgs])
    x_test=np.array(imgs[(k*testSetSize):((k+1)*testSetSize)])
    return([x_train,x_test])

y_train=np.array([np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]*560)
y_test=np.array([np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]*140)

In [26]:
#data for binary segmentation
posPatients = np.concatenate((np.array([2,4,5,7,9,11,12,13,16,17,18,19,21,22]),range(23,52),np.array([55]),
                                range(60,63),range(64,74),range(75,82),range(83,104),range(105,109)))
locList = []
maskList = []
posList = []
img_height=128
limit=0.25
pixLimit=5

for i in range(len(posPatients)):
    if posPatients[i]<23:
        if posPatients[i]==12:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/12_anon/12_maski/12_maskit.img'
        else:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}_anon/{}_maski/{}_maski.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    else:
        if posPatients[i]==23:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/23/23_maski/23_nifti_maski_edit.img'
        if posPatients[i]==24:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/24/24_maski/24_maski.img'
        if posPatients[i]==38:
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/38/38_nifti_maski/38_nifti_maski_edit.img'
        if posPatients[i] not in [23,24,38]:    
            img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}/{}_nifti_maski/{}_nifti_maski.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    img = nib.load(img_path)
    img_data = img.get_fdata()
    locations = []
    masks = []
    for j in range(img_data.shape[2]):
        slice = np.array(img_data[:,:,j])
        if np.max(slice)>0:
            slice=cv2.resize(slice,(img_height,img_height))
            for k in range(img_height):
                for l in range(img_height):
                    if slice[k,l]>limit:
                        slice[k,l]=1
                    else:
                        slice[k,l]=0
            if np.sum(slice)>pixLimit:
                locations.append(j)
                masks.append(slice)
    locList.append(np.array(locations))
    maskList.append(masks)

for i in range(len(posPatients)):
    if posPatients[i]<23:
        img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}_anon/{}_nifti_pet/{}_nifti_pet.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    else:
        img_path = 'C:/Users/Oona/Documents/Tpc/hnc/anom_data/positiiviset/{}/{}_nifti_pet/{}_nifti_pet.img'.format(
                        posPatients[i],posPatients[i],posPatients[i])
    img = nib.load(img_path)
    img_data = img.get_fdata()
    pos = []
    for j in range(len(locList[i])):
        pos.append(cv2.resize(np.array(img_data[:,:,locList[i][j]]),(img_height,img_height)))
    posList.append(pos)

def convert(img):

    img=(img-np.min(img))/(np.max(img)-np.min(img))*255
    for i in range(img_height):
        for j in range(img_height):
            img[i,j]=int(img[i,j])
    return img.astype('int64')

for i in range(len(posList)):
    for j in range(len(posList[i])):
        posList[i][j]=convert(posList[i][j])

numberOfSlices=[]
finPatients=0
for i in range(len(posList)):
    numberOfSlices.append(len(posList[i]))
    if len(posList[i])>0:
        finPatients+=1
numberOfSlices=np.array(numberOfSlices)
print(finPatients)
print(sum(numberOfSlices))
print(np.mean(numberOfSlices))
print(np.std(numberOfSlices))
print(numberOfSlices)

testSetPatients=np.concatenate((range(0,70,5),np.array([72,82,83])))

numTestSetSlices=0
numTestSetPatients=0
for i in range(len(posList)):
    if i in testSetPatients:
        numTestSetSlices+=len(posList[i])
        if len(posList[i])>0:
            numTestSetPatients+=1

print(numTestSetSlices)
print(numTestSetSlices/sum(numberOfSlices))
print(numTestSetPatients)

x_train = []
y_train = []
x_test = []
y_test = []

for i in range(len(posList)):
    if i in testSetPatients:
        for j in range(len(posList[i])):
            img=(posList[i][j]-np.min(posList[i][j]))/(np.max(posList[i][j])-np.min(posList[i][j]))
            x_test.append(img)
            y_test.append(cv2.resize(maskList[i][j],(img_height,img_height)))   
    else:
        for j in range(len(posList[i])):
            img=(posList[i][j]-np.min(posList[i][j]))/(np.max(posList[i][j])-np.min(posList[i][j]))
            x_train.append(img)
            y_train.append(cv2.resize(maskList[i][j],(img_height,img_height)))  

x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)
y_test=np.array(y_test)

89
962
10.808988764044944
6.063107084385669
[ 3  9 11 14  8  9 15 12  5 10 13  8 11 10  4 13  4  4  6  4  6  6  5 12
 12  6 18 18  6 19  7 12 13 22 10  5 20  5 12 25  5  7  2  9  6 18 24 11
 11 13 21  7  6  9 13  5  2  1  7  3 18 15  6 20 26 16 13 10 11  6  3  4
 15  6 26  6 10 10 20  5 17 13 18 13 14 11  4 11 23]
191
0.19854469854469856
17


In [10]:
#convolutional neural networks

def unetForBinClass(x_train,y_train,x_test,numberOfEpochs):

    img_height=x_train[0].shape[0]
    model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(img_height,img_height,1)),
                                    tf.keras.layers.Conv2D(16, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(32, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(32, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(64, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(64, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(128, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(128, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(strides=(2, 2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dense(32, activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
    ] 
    )
    model.compile(
        optimizer=tf.keras.optimizers.SGD(1e-3),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,validation_split=0.3,shuffle=True)
    predictions=model.predict(x_test)
    trainPreds=model.predict(x_train)
    return([predictions,trainPreds])

def inceptionForBinClass(x_train,y_train,x_test,numEpochs):
    
    img_height=x_train[0].shape[0]
    if len(x_train.shape)==3:
        x_train=np.rollaxis(np.array([x_train,x_train,x_train]),0,4)
        x_test=np.rollaxis(np.array([x_test,x_test,x_test]),0,4)
    ntf_model = keras.applications.InceptionV3(weights=None,input_shape=(img_height,img_height,3),include_top=False)
    ntf_model.trainable = True
    inputs = keras.Input(shape=(img_height,img_height,3))
    x = ntf_model(inputs, training=True)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(1)(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.SGD(1e-3),
                loss=keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=[keras.metrics.BinaryAccuracy()])
    model.fit(x=x_train, y=y_train, epochs=numEpochs, validation_split=0.3, shuffle=True)
    ntfPredictions=model.predict(x_test)
    ntfTrainPreds=model.predict(x_train)
    return([ntfPredictions, ntfTrainPreds])

def unetForMultiClass(x_train,y_train,x_test,numberOfEpochs):

    img_height=x_train[0].shape[0]
    model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16, 3, activation='relu', input_shape=(img_height,img_height,1)),
                                    tf.keras.layers.Conv2D(16, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(32, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(32, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(64, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(64, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
                                    tf.keras.layers.Conv2D(128, 3, activation='relu'),
                                    tf.keras.layers.Conv2D(128, 3, activation='relu'),
                                    tf.keras.layers.MaxPooling2D(strides=(2, 2)),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(128, activation='relu'),
                                    tf.keras.layers.Dense(64, activation='relu'),
                                    tf.keras.layers.Dense(32, activation='relu'),
                                    tf.keras.layers.Dense(4, activation='softmax')
    ] 
    )
    model.compile(
        optimizer=tf.keras.optimizers.SGD(1e-3),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=[tf.keras.metrics.CategoricalAccuracy()]
    )
    model.fit(x=x_train,y=y_train,epochs=numberOfEpochs,validation_split=0.3,shuffle=True)
    predictions=model.predict(x_test)
    trainPreds=model.predict(x_train)
    return([predictions,trainPreds])

def inceptionForMultiClass(x_train,y_train,x_test,numEpochs):
    
    img_height=x_train[0].shape[0]
    if len(x_train.shape)==3:
        x_train=np.rollaxis(np.array([x_train,x_train,x_train]),0,4)
        x_test=np.rollaxis(np.array([x_test,x_test,x_test]),0,4)
    ntf_model = keras.applications.InceptionV3(weights=None,input_shape=(img_height,img_height,3),include_top=False)
    ntf_model.trainable = True
    inputs = keras.Input(shape=(img_height,img_height,3))
    x = ntf_model(inputs, training=True)
    x = keras.layers.GlobalAveragePooling2D()(x)
    outputs = keras.layers.Dense(4,activation='softmax')(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer=keras.optimizers.SGD(1e-3),
                loss=keras.losses.CategoricalCrossentropy(),
                metrics=[keras.metrics.CategoricalAccuracy()])
    model.fit(x=x_train, y=y_train, epochs=numEpochs, validation_split=0.3, shuffle=True)
    ntfPredictions=model.predict(x_test)
    ntfTrainPreds=model.predict(x_train)
    return([ntfPredictions, ntfTrainPreds])

def unetForSegmentation(x_train,y_train,x_test,numEpochs):

    img_height=x_train[0].shape[0]

    #U-Net model
    inputs = tf.keras.layers.Input(shape=(img_height,img_height,1))

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)

    #Expansive path
    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c3)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='linear')(c9) #sigmoid or linear

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    model.fit(x=x_train,y=y_train,epochs=numEpochs,validation_split=0.3,shuffle=True)
    trainPreds=model.predict(x_train)
    predictions=model.predict(x_test)
    return([predictions,trainPreds])

def deeperUnetForSegmentation(x_train,y_train,x_test,numEpochs):

    img_height=x_train[0].shape[0]

    #U-Net model
    inputs = tf.keras.layers.Input(shape=(img_height,img_height,1))

    #Contraction path
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
    c1 = tf.keras.layers.Dropout(0.1)(c1)
    c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
    c2 = tf.keras.layers.Dropout(0.1)(c2)
    c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
    c3 = tf.keras.layers.Dropout(0.2)(c3)
    c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
    c4 = tf.keras.layers.Dropout(0.2)(c4)
    c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)

    #Expansive path

    u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c4)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
    c7 = tf.keras.layers.Dropout(0.1)(c7)
    c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
    c8 = tf.keras.layers.Dropout(0.1)(c8)
    c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
    c9 = tf.keras.layers.Dropout(0.1)(c9)
    c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1,1), activation='linear')(c9) #sigmoid or linear

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    model.compile(optimizer='adam', loss='binary_crossentropy')
    model.fit(x=x_train,y=y_train,epochs=numEpochs,validation_split=0.3,shuffle=True)
    trainPreds=model.predict(x_train)
    predictions=model.predict(x_test)
    return([predictions,trainPreds])

In [ ]:
predictions,trainPreds=unetForBinClass(x_train,y_train,x_test,10)
np.savetxt('unet_binClass_trainPreds.txt',trainPreds)
np.savetxt('unet_binClass_predictions.txt',predictions)
predictions,trainPreds=inceptionForBinClass(x_train,y_train,x_test,10)
np.savetxt('inception_binClass_trainPreds.txt',trainPreds)
np.savetxt('inception_binClass_predictions.txt',predictions)

In [ ]:
for k in range(5):
    x_train,x_test=divideImgsIntoSets(imgs,k)
    for j in range(5):
        predictions,trainPreds=unetForMultiClass(x_train,y_train,x_test,10)
        np.savetxt('unet_multiClass_predictions_{}_{}.txt'.format(k,j),predictions)
        predictions,trainPreds=inceptionForMultiClass(x_train,y_train,x_test,10)
        np.savetxt('inception_multiClass_predictions_{}_{}.txt'.format(k,j),predictions)

In [14]:
predictions,trainPreds=unetForSegmentation(x_train,y_train,x_test,50)
for i in range(len(trainPreds)):
    np.savetxt('unet_segmentation_trainPreds_{}.txt'.format(i), trainPreds[i][:,:,0])
for i in range(len(predictions)):
    np.savetxt('unet_segmentation_predictions_{}.txt'.format(i), predictions[i][:,:,0])
predictions,trainPreds=deeperUnetForSegmentation(x_train,y_train,x_test,50)
for i in range(len(trainPreds)):
    np.savetxt('deep_segmentation_trainPreds_{}.txt'.format(i), trainPreds[i][:,:,0])
for i in range(len(predictions)):
    np.savetxt('deep_segmentation_predictions_{}.txt'.format(i), predictions[i][:,:,0])

In [7]:
#functions for evaluation metrics

def evaluateBinClassPredictions(predictions,y_test,trainPreds,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    youden = thresholds[np.argmax(tpr - fpr)]
    TN = 0
    FN = 0
    TP = 0
    FP = 0
    for i in range(len(y_test)):
        if predictions[i]<youden:
            if y_test[i]==0:
                TN+=1
            else:
                FN+=1
        else:
            if y_test[i]==1:
                TP+=1
            else:
                FP+=1
    acc = (TN+TP)/(TN+FN+TP+FP)
    sen = TP/(FN+TP)
    spe = TN/(TN+FP)
    pre = TP/(FP+TP)
    youden = sen+spe-1
    f1 = 2*pre*sen/(pre+sen)
    p_e=((TP+FN)*(TP+FP)+(TN+FP)*(TN+FN))/(TP+TN+FN+FP)**2
    kappa=(acc-p_e)/(1-p_e)
    mcc=(TN*TP-FN*FP)/((TP+FN)*(TP+FP)*(TN+FP)*(TN+FN))**0.5
    fpr, tpr, thresholds = roc_curve(y_test, predictions, drop_intermediate=False)
    auc1 = auc(fpr,tpr)
    return([acc,sen,spe,pre,youden,f1,kappa,mcc,auc1])

def tablesForMcNemar(predictions,trainPreds,predictions1,trainPreds1,y_test,y_train):

    fpr, tpr, thresholds = roc_curve(y_train, trainPreds, drop_intermediate=False)
    youden = thresholds[np.argmax(tpr - fpr)]
    fpr, tpr, thresholds = roc_curve(y_train, trainPreds1, drop_intermediate=False)
    youden1 = thresholds[np.argmax(tpr - fpr)]
    negTable=np.zeros((2,2))
    posTable=np.zeros((2,2))
    for i in range(len(y_test)):
        if y_test[i]==1:
            posTable[int(predictions[i]<youden),int(predictions1[i]<youden1)]+=1
        else:
            negTable[int(predictions[i]<youden),int(predictions1[i]<youden1)]+=1
    return([posTable,negTable])

def evaluateMultiClassPredictions(predictions,y_test):

    n=len(predictions[0])
    confMatrix=np.zeros((n,n))
    for i in range(len(predictions)):
        confMatrix[np.argmax(y_test[i]),np.argmax(predictions[i])]+=1
    counts=np.zeros((n,4))
    macroAverageMetrics=np.zeros((6))
    p_e=0
    c_0=0
    c_1=0
    for i in range(n):
        TP=confMatrix[i,i]
        FN=np.sum(confMatrix[i,:])-TP
        FP=np.sum(confMatrix[:,i])-TP
        TN=np.sum(confMatrix)-TP-FN-FP
        counts[i,:]=[TP,FN,FP,TN]
        acc = (TN+TP)/(TN+FN+TP+FP)
        sen = TP/(FN+TP)
        spe = TN/(TN+FP)
        pre = TP/(FP+TP)
        youden = sen+spe-1
        f1 = 2*pre*sen/(pre+sen)
        macroAverageMetrics+=[acc,sen,spe,pre,youden,f1]
        p_e+=np.sum(confMatrix[i,:])*np.sum(confMatrix[:,i])/np.sum(confMatrix)**2
        c_0+=np.sum(confMatrix[i,:])**2/np.sum(confMatrix)**2
        c_1+=np.sum(confMatrix[:,i])**2/np.sum(confMatrix)**2
    macroAverageMetrics=macroAverageMetrics/n
    TP=np.sum(counts[:,0])
    FN=np.sum(counts[:,1])
    FP=np.sum(counts[:,2])
    TN=np.sum(counts[:,3])
    acc = (TN+TP)/(TN+FN+TP+FP)
    sen = TP/(FN+TP)
    spe = TN/(TN+FP)
    pre = TP/(FP+TP)
    youden = sen+spe-1
    f1 = 2*pre*sen/(pre+sen)
    microAverageMetrics=np.array([acc,sen,spe,pre,youden,f1])
    p_0=TP/np.sum(confMatrix)
    kappa=(p_0-p_e)/(1-p_e)
    mcc=(p_0-p_e)/((1-c_0)*(1-c_1))**0.5
    return([macroAverageMetrics,microAverageMetrics,kappa,mcc])

def dicesForLimit(predictions,y_test,limit):

    dices=np.zeros((len(y_test)))
    ious=np.zeros((len(y_test)))
    img_height=predictions[0].shape[0]
    for k in range(len(y_test)):
        TP=0
        FN=0
        FP=0
        for i in range(img_height):
            for j in range(img_height):
                if y_test[k,i,j]>0.5 and predictions[k,i,j]>limit:
                    TP+=1
                if y_test[k,i,j]>0.5 and predictions[k,i,j]<limit:
                    FN+=1
                if y_test[k,i,j]<0.5 and predictions[k,i,j]>limit:
                    FP+=1
        if TP+FN+FP>0:
            dices[k]=2*TP/(2*TP+FN+FP)
            ious[k]=TP/(TP+FN+FP)
    return([dices,ious])

def turnIntoBinaryMask(img,lim):

    img_height=img.shape[0]
    binM=np.zeros((img_height,img_height))
    for i in range(img_height):
        for j in range(img_height):
            if img[i][j]>lim:
                binM[i][j]=1
    return binM

In [18]:
unetTrainPreds=np.loadtxt('unet_binClass_trainPreds.txt')
unetPreds=np.loadtxt('unet_binClass_predictions.txt')
incTrainPreds=np.loadtxt('inception_binClass_trainPreds.txt')
incPreds=np.loadtxt('inception_binClass_predictions.txt')
y_test=np.array([0,1]*int(unetPreds.shape[0]/2))
y_train=np.array([0,1]*int(unetTrainPreds.shape[0]/2))
print(evaluateBinClassPredictions(unetPreds,y_test,unetTrainPreds,y_train))
print(evaluateBinClassPredictions(incPreds,y_test,incTrainPreds,y_train))
posTable,negTable=tablesForMcNemar(unetPreds,unetTrainPreds,incPreds,incTrainPreds,y_test,y_train)
print(posTable)
print(mcnemar(posTable))
print(negTable)
print(mcnemar(negTable))

[0.7566666666666667, 0.87, 0.6433333333333333, 0.7092391304347826, 0.5133333333333332, 0.781437125748503, 0.5133333333333334, 0.5270512152611412, 0.8453888888888889]
[0.7316666666666667, 0.7533333333333333, 0.71, 0.7220447284345048, 0.4633333333333334, 0.737357259380098, 0.4633333333333334, 0.46376896546319546, 0.8207666666666666]
[[207.  54.]
 [ 19.  20.]]
pvalue      5.0622659111780655e-05
statistic   19.0
[[ 63.  44.]
 [ 24. 169.]]
pvalue      0.02052693371370707
statistic   24.0


In [8]:
array=np.zeros((25,14))
array1=np.zeros((25,14))
y_test=np.array([np.array([1,0,0,0]),np.array([0,1,0,0]),np.array([0,0,1,0]),np.array([0,0,0,1])]*140)
for k in range(5):
    for j in range(5):
        predictions=np.loadtxt('unet_multiClass_predictions_{}_{}.txt'.format(k,j))
        macroAverageMetrics,microAverageMetrics,kappa,mcc=evaluateMultiClassPredictions(predictions,y_test)
        array[5*k+j,0:6]=macroAverageMetrics
        array[5*k+j,6:12]=microAverageMetrics
        array[5*k+j,12]=kappa
        array[5*k+j,13]=mcc
        predictions=np.loadtxt('inception_multiClass_predictions_{}_{}.txt'.format(k,j))
        macroAverageMetrics,microAverageMetrics,kappa,mcc=evaluateMultiClassPredictions(predictions,y_test)
        array1[5*k+j,0:6]=macroAverageMetrics
        array1[5*k+j,6:12]=microAverageMetrics
        array1[5*k+j,12]=kappa
        array1[5*k+j,13]=mcc
print(np.median(array,axis=0))
print(np.median(array1,axis=0))

[0.85714286 0.71428571 0.9047619  0.75968604 0.61904762 0.69319037
 0.85714286 0.71428571 0.9047619  0.71428571 0.61904762 0.71428571
 0.61904762 0.64106434]
[0.80089286 0.60178571 0.8672619  0.58915493 0.46904762 0.59066675
 0.80089286 0.60178571 0.8672619  0.60178571 0.46904762 0.60178571
 0.46904762 0.47252601]


In [13]:
for i in range(14):
    print(i)
    print(round(np.median(array[:,i]),3))
    print(round(np.median(array1[:,i]),3))
    print(ttest_rel(array[:,i],array1[:,i]))
    print(wilcoxon(array[:,i],array1[:,i]))

0
0.857
0.801
Ttest_relResult(statistic=5.221358637324015, pvalue=2.3749483100553514e-05)
WilcoxonResult(statistic=25.0, pvalue=5.3882598876953125e-05)
1
0.714
0.602
Ttest_relResult(statistic=5.221358637324018, pvalue=2.3749483100553335e-05)
WilcoxonResult(statistic=25.5, pvalue=6.365776062011719e-05)
2
0.905
0.867
Ttest_relResult(statistic=5.221358637324022, pvalue=2.3749483100553074e-05)
WilcoxonResult(statistic=24.5, pvalue=5.3882598876953125e-05)
3
0.76
0.589
Ttest_relResult(statistic=7.999513823423033, pvalue=3.1593973183628113e-08)
WilcoxonResult(statistic=4.0, pvalue=4.172325134277344e-07)
4
0.619
0.469
Ttest_relResult(statistic=5.221358637324019, pvalue=2.3749483100553294e-05)
WilcoxonResult(statistic=25.5, pvalue=6.365776062011719e-05)
5
0.693
0.591
Ttest_relResult(statistic=3.9212486047391217, pvalue=0.0006427747665956361)
WilcoxonResult(statistic=47.0, pvalue=0.0011548399925231934)
6
0.857
0.801
Ttest_relResult(statistic=5.221358637324019, pvalue=2.3749483100553294e-05)
Wilc

In [9]:
k=j=0
predictions=np.loadtxt('unet_multiClass_predictions_{}_{}.txt'.format(k,j))
n=len(predictions[0])
confMatrix=np.zeros((n,n))
for i in range(len(predictions)):
    confMatrix[np.argmax(y_test[i]),np.argmax(predictions[i])]+=1
print(confMatrix)
print(evaluateMultiClassPredictions(predictions,y_test))

[[120.   7.   9.   4.]
 [ 15. 116.   3.   6.]
 [ 12.  13. 115.   0.]
 [  2.  96.   4.  38.]]
[array([0.84732143, 0.69464286, 0.89821429, 0.7437246 , 0.59285714,
       0.67676739]), array([0.84732143, 0.69464286, 0.89821429, 0.69464286, 0.59285714,
       0.69464286]), 0.5928571428571429, 0.6156458078077206]


In [27]:
trainPreds=[]
predictions=[]
trainPreds1=[]
predictions1=[]
for i in range(len(y_train)):
    trainPreds.append(np.loadtxt('unet_segmentation_trainPreds_{}.txt'.format(i)))
for i in range(len(y_test)):
    predictions.append(np.loadtxt('unet_segmentation_predictions_{}.txt'.format(i)))
for i in range(len(y_train)):
    trainPreds1.append(np.loadtxt('deep_segmentation_trainPreds_{}.txt'.format(i)))
for i in range(len(y_test)):
    predictions1.append(np.loadtxt('deep_segmentation_predictions_{}.txt'.format(i)))
trainPreds=np.array(trainPreds)
predictions=np.array(predictions)
trainPreds1=np.array(trainPreds1)
predictions1=np.array(predictions1)

In [45]:
np.median(dicesForLimit(trainPreds1,y_train,0.44)[0])
#0.15 0.605, 0.17 0.613, 0.19 0.615, 0.2 0.618, 0.21 0.614, 0.25 0.595, 0.3 0.56
#0.2 0.611, 0.25 0.648, 0.3 0.688, 0.35 0.720, 0.4 0.727, 0.43 0.722, 0.44 0.724, 0.45 0.730, 0.46 0.728, 0.5 0.713

0.7241379310344828

In [62]:
limit=0.2
limit1=0.45
dices,ious=dicesForLimit(predictions,y_test,limit)
dices1,ious1=dicesForLimit(predictions1,y_test,limit1)

In [ ]:
def f_test(x,y):

    x=np.array(x)
    y=np.array(y)
    f=np.var(x,ddof=1)/np.var(y,ddof=1)
    dfn=x.size-1
    dfd=y.size-1
    p=2*min(1-scipy.stats.f.cdf(f,dfn,dfd),scipy.stats.f.cdf(f,dfn,dfd))
    return(p)

In [59]:
x=ious
y=ious1
print(round(np.median(x),3))
print(round(np.median(y),3))
print(ttest_rel(x,y))
print(wilcoxon(x,y))
print(round(np.std(x),3))
print(round(np.std(y),3))
print(shapiro(x))
print(shapiro(y))
print(f_test(x,y))
print(bartlett(x,y))
print(levene(x,y))

0.32
0.403
Ttest_relResult(statistic=-4.456370034222895, pvalue=1.4221215233273695e-05)
WilcoxonResult(statistic=4428.0, pvalue=6.436702840509736e-06)
0.259
0.285
ShapiroResult(statistic=0.9345442652702332, pvalue=1.3804864806843398e-07)
ShapiroResult(statistic=0.9234173893928528, pvalue=1.9048348676165006e-08)
0.1822258953078349
BartlettResult(statistic=1.7793790527405136, pvalue=0.18222559367108443)
LeveneResult(statistic=4.537676241931248, pvalue=0.03379843047166618)


In [61]:
def get_all_edges(bool_img):
    """
    Get a list of all edges (where the value changes from True to False) in the 2D boolean image.
    The returned array edges has he dimension (n, 2, 2).
    Edge i connects the pixels edges[i, 0, :] and edges[i, 1, :].
    Note that the indices of a pixel also denote the coordinates of its lower left corner.
    """
    edges = []
    ii, jj = np.nonzero(bool_img)
    for i, j in zip(ii, jj):
        # North
        if j == bool_img.shape[1]-1 or not bool_img[i, j+1]:
            edges.append(np.array([[i, j+1],
                                   [i+1, j+1]]))
        # East
        if i == bool_img.shape[0]-1 or not bool_img[i+1, j]:
            edges.append(np.array([[i+1, j],
                                   [i+1, j+1]]))
        # South
        if j == 0 or not bool_img[i, j-1]:
            edges.append(np.array([[i, j],
                                   [i+1, j]]))
        # West
        if i == 0 or not bool_img[i-1, j]:
            edges.append(np.array([[i, j],
                                   [i, j+1]]))

    if not edges:
        return np.zeros((0, 2, 2))
    else:
        return np.array(edges)


def close_loop_edges(edges):
    """
    Combine the edges defined by 'get_all_edges' to closed loops around objects.
    If there are multiple disconnected objects a list of closed loops is returned.
    Note that it's expected that all the edges are part of exactly one loop (but not necessarily the same one).
    """

    loop_list = []
    while edges.size != 0:

        loop = [edges[0, 0], edges[0, 1]]  # Start with first edge
        edges = np.delete(edges, 0, axis=0)

        while edges.size != 0:
            # Get next edge (=edge with common node)
            ij = np.nonzero((edges == loop[-1]).all(axis=2))
            if ij[0].size > 0:
                i = ij[0][0]
                j = ij[1][0]
            else:
                loop.append(loop[0])
                # Uncomment to to make the start of the loop invisible when plotting
                # loop.append(loop[1])
                break

            loop.append(edges[i, (j + 1) % 2, :])
            edges = np.delete(edges, i, axis=0)

        loop_list.append(np.array(loop))

    return loop_list


def plot_outlines(bool_img, ax=None, **kwargs):
    if ax is None:
        ax = plt.gca()

    edges = get_all_edges(bool_img=bool_img)
    edges = edges - 0.5  # convert indices to coordinates; TODO adjust according to image extent
    outlines = close_loop_edges(edges=edges)
    cl = LineCollection(outlines, **kwargs)
    ax.add_collection(cl)

In [ ]:
i=8
print(i,round(dices1[i],3),round(ious1[i],3))
array=cv2.rotate(y_test[i],cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
array1=cv2.rotate(turnIntoBinaryMask(predictions1[i],limit1),cv2.cv2.ROTATE_90_COUNTERCLOCKWISE)
plt.imshow(array1)
plt.imshow(cv2.rotate(x_test[i],cv2.cv2.ROTATE_90_COUNTERCLOCKWISE), cmap='gray')
fig=plt.gcf()
plot_outlines(array.T, lw=2, color='white')
plot_outlines(array1.T, lw=2, color='blue')
plt.axis('off')
plt.savefig('fig{}.png'.format(i),bbox_inches='tight')
plt.show()

In [111]:
img_height=128
k=8
TP=0
FN=0
FP=0
TN=0
for i in range(img_height):
    for j in range(img_height):
        if y_test[k,i,j]>0.5 and predictions1[k,i,j]>limit1:
            TP+=1
        if y_test[k,i,j]>0.5 and predictions1[k,i,j]<limit1:
            FN+=1
        if y_test[k,i,j]<0.5 and predictions1[k,i,j]>limit1:
            FP+=1
        if y_test[k,i,j]<0.5 and predictions1[k,i,j]<limit1:
            TN+=1
print(TP,TN,FP,FN)

181 16156 17 30


In [112]:
print(2*TP/(2*TP+FN+FP))
print(TP/(TP+FN+FP))
print((TP+TN)/(TP+TN+FN+FP))

0.8850855745721271
0.793859649122807
0.99713134765625
